In [1]:
# third party
import sys
import matplotlib.pyplot as plt
import numpy as np
import sys
from pathlib import Path
import psyplot.project as psy
import pandas as pd
import xarray
from netCDF4 import Dataset,date2num
import metpy.calc as calc
from metpy.units import units
import datetime as dt
import pandas as pd
from iconarray.plot import formatoptions # import plotting formatoptions (for use with psyplot)
import iconarray as iconvis # import self-written modules from iconarray

# first party
sys.path.append('../utilities_tlezuo/')
from timefunctions import *
import varfunctions as vf
import locfunctions as lf

INFO:numexpr.utils:Note: NumExpr detected 36 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
###############################################################################################
## CONSTANT FILE  ##
c_filename = "lfff00000000c.nc"
filepath = '/store/s83/swester/teamx/tdf_2019091212/output/19091212/'
const_file = filepath + c_filename
data_c = psy.open_dataset(const_file)
# height loading
HFL = (data_c['HHL'][1:].values + (data_c['HHL'][:-1].values-data_c['HHL'][1:].values)/2)
HHL = data_c['HHL'].values
HSURF = data_c['HSURF'].values

###############################################################################################
## DECIDE ##
# where to save
savepath = '/users/tlezuo/icon-vis/height_time_diagrams/data/'

# LOCATIONS
loc_list=[lf.hoch,lf.kols,lf.egg,lf.weer,lf.terf,lf.arb,lf.murs,lf.ifl]

# VARIABLES
# 3D
pvars_list= [vf.U, vf.V, vf.T, vf.QV,vf.P,vf.W,vf.TKE]
cvars_list = [vf.VEL, vf.DIR,vf.TD,vf.RH,vf.TH]
# 2d surface
spvars_list = [vf.T_2M,vf.QV_2M,vf.U_10M,vf.V_10M]
scvars_list = [vf.VEL_10M, vf.DIR_10M,vf.TKEs]

# TIME
startdate = dt.datetime(2019,9,12,12,00)
enddate = dt.datetime(2019,9,14,00,00)

plotfreq = '0h30min'
simdate = dt.datetime(2019,9,12,12,00) # no change,. simulation start
plotdates = pd.date_range(startdate,enddate,freq=plotfreq)
## TEST ##
plotdate = dt.datetime(2019,9,13,8,30)
simdate = dt.datetime(2019,9,12,12,00)
lt = get_lt(plotdate,simdate)

print(lt)
filename = lfff_name(lt)
nc_file = '/store/s83/swester/teamx/tdf_2019091212/output/19091212/'+filename
# If necessary, add the corresponding grid file:
grid_file = '../data/example_data/grids/ICON-1E_DOM01.nc'

if iconvis.check_grid_information(nc_file):
    print('The grid information is available')
    data = psy.open_dataset(nc_file)
else:
    print('The grid information is not available')
    data = iconvis.combine_grid_information(nc_file,grid_file)
print(filename)

20.5
The grid information is available
lfff00203000.nc


## OLD SETUP ##
# loop 1 over locations
for loc in loc_list:
    # get closest point
    lats = np.rad2deg(data.clat.values[:])
    lons = np.rad2deg(data.clon.values[:])
    ind = iconvis.ind_from_latlon(lats,lons,loc.lat,loc.lon,verbose=True)

    # initialize lidar dataset for this location
    lidar_data = {}
    for pvar in pvars_list:
        lidar_data[pvar.name] = np.zeros((len(plotdates),80)) 
    lidar_data['W'] = np.zeros((len(plotdates),81)) # 81 levels for halflevel-variables as W
    lidar_data['TKE'] = np.zeros((len(plotdates),81)) # 81 levels for halflevel-variables as W
    for cvar in cvars_list:
        lidar_data[cvar.name] = np.zeros((len(plotdates),80))
    
    # initialize ts dataset for this location
    ts_data = {}
    for spvar in spvars_list:
        ts_data[spvar.name] = np.zeros(len(plotdates)) 
    for scvar in scvars_list:
        ts_data[scvar.name] = np.zeros(len(plotdates)) 

    # get height coordinates in meter for this location
    lidar_data['HFL_loc'] = HFL[:,ind][::-1] # height full levels
    lidar_data['HHL_loc'] = HHL[:,ind][::-1] # height half levels
    # levels
    lidar_data['height'] = data['height_3'].data.squeeze()[::-1] # reverse the height to have 81 at bottom

    # loop 2 over time
    for counter,pdate in enumerate(plotdates):
        print('    calculating lidar and timeseries at ' + loc.name + ' '+pdate.strftime('%Y/%m/%d at %H:%M'))
        ## READ IN ##
        lt = get_lt(pdate,simdate) # get leadtime
        filename = lfff_name(lt) # get filename
        nc_file = filepath+filename
        # If necessary, add the corresponding grid file:
        grid_file = '/store/s83/swester/teamx/grid_R19B08_size_cosmo1/d01_DOM01.nc'

        if iconvis.check_grid_information(nc_file):
            # print('             The grid information is available')
            data = psy.open_dataset(nc_file)
        else:
            print('             The grid information is not available')
            data = iconvis.combine_grid_information(nc_file,grid_file)

        # select variables from data dataset
        for pvar in pvars_list: # 3D vars for lidar
            # print(pvar.name)
            lidar_data[pvar.name][counter] = data[pvar.name][0,:,ind].data.squeeze()[::-1] # reverse the variable arrays to have L 81 data at bottom
        for spvar in spvars_list: # 2D vars for lidar
            # print(pvar.name)
            ts_data[spvar.name][counter] = data[spvar.name].data.squeeze()[ind]
       
        ## CALCULATE NEW VARS ##
        for cvar in cvars_list:
            # VEL
            if cvar.name == 'VEL':
                lidar_data[cvar.name][counter] = vf.calculate_wind_vel_from_uv(lidar_data['U'][counter],lidar_data['V'][counter])
                # lidar_data[cvar.name][counter] = lidar_data[cvar.name][counter][::-1]
            # DIR
            elif cvar.name == 'DIR':
                lidar_data[cvar.name][counter] = vf.calculate_wind_dir_from_uv(lidar_data['U'][counter],lidar_data['V'][counter],modulo_180=False)
            # TH 
            elif cvar.name == 'TH':
                lidar_data[cvar.name][counter] = vf.calculate_potT(lidar_data['T'][counter],lidar_data['P'][counter])
            # TD
            elif cvar.name == 'TD':
                lidar_data[cvar.name][counter] = calc.dewpoint_from_specific_humidity(lidar_data['P'][counter]* units.hPa, lidar_data['T'][counter]* units.kelvin, lidar_data['QV'][counter]* units('g/kg'))
             # RH
            elif cvar.name == 'RH':
                lidar_data[cvar.name][counter] = calc.relative_humidity_from_specific_humidity(lidar_data['P'][counter]* units.hPa, lidar_data['T'][counter]* units.kelvin, lidar_data['QV'][counter]* units('g/kg'))
        
        # 2D surface
        for scvar in scvars_list:
            # VEL 10m
            if scvar.name == 'VEL_10M':
                ts_data[scvar.name][counter] = vf.calculate_wind_vel_from_uv(ts_data['u_10m'][counter],ts_data['v_10m'][counter])
            # DIR 10m
            elif scvar.name == 'DIR_10M':
                ts_data[scvar.name][counter] = vf.calculate_wind_dir_from_uv(ts_data['u_10m'][counter],ts_data['v_10m'][counter],modulo_180=False)

    ## SAVE lidar/htd data ##
    np.save(savepath+'htd_ICON_'+loc.name+'.npy', lidar_data) 
    ## SAVE timeseries data ##
    np.save(savepath+'ts_ICON_'+loc.name+'.npy', ts_data) 

In [3]:
# initialize dicts 
lidar_data = {}
ts_data = {}

for loc in loc_list:
    # initialize dicts for this location
    lidar_data[loc.name] = {}
    ts_data[loc.name] = {}

    # initialize dicts for every variable
    for pvar in pvars_list:
        lidar_data[loc.name][pvar.name] = np.zeros((len(plotdates),80)) 
    lidar_data[loc.name]['W'] = np.zeros((len(plotdates),81)) # 81 levels for halflevel-variables as W
    lidar_data[loc.name]['TKE'] = np.zeros((len(plotdates),81)) # 81 levels for halflevel-variables as W
    for cvar in cvars_list:
        lidar_data[loc.name][cvar.name] = np.zeros((len(plotdates),80))

    for spvar in spvars_list:
        ts_data[loc.name][spvar.name] = np.zeros(len(plotdates)) 
    for scvar in scvars_list:
        ts_data[loc.name][scvar.name] = np.zeros(len(plotdates)) 

    # get height coordinates in meter for this location
    lidar_data[loc.name]['HFL_loc'] = HFL[:,loc.iconID][::-1] # height full levels
    lidar_data[loc.name]['HHL_loc'] = HHL[:,loc.iconID][::-1] # height half levels
    lidar_data[loc.name]['HSURF_loc'] = HSURF[loc.iconID] # topography model
    # levels
    lidar_data[loc.name]['height'] = data['height_3'].data.squeeze()[::-1] # reverse the height to have 81 at bottom


In [4]:
## GET icon gridpoint id to include it in the lf.loc.iconID for new stations
for loc in loc_list:
    lats = np.rad2deg(data.clat.values[:])
    lons = np.rad2deg(data.clon.values[:])
    ind = iconvis.ind_from_latlon(lats,lons,loc.lat,loc.lon,verbose=True)
    print('the grid index for '+loc.name+' is '+str(ind))
    print('the lowest level has a height of half level of '+str(np.round(lidar_data[loc.name]['HHL_loc'][0])))
    print('the lowest level has a height of full level of '+str(np.round(lidar_data[loc.name]['HFL_loc'][0])))
    print('the model topography is '+str(np.round(lidar_data[loc.name]['HSURF_loc'])))
    print('the coordinates for this ind are lat= '+str(np.rad2deg(data_c.clat[ind].values))+' lon= '+str(np.rad2deg(data_c.clon[ind].values)))
    print('the real coordinates for this point are lat= '+str(loc.lat)+' lon= '+str(loc.lon))

Closest ind: 810547
 Given lat: 47.288 vs found lat: 47.284
 Given lon: 11.631 vs found lon: 11.632
the grid index for Hochhaueser is 810547
the lowest level has a height of half level of 1124.0
the lowest level has a height of full level of 1134.0
the model topography is 1124.0
the coordinates for this ind are lat= 47.28385 lon= 11.63243
the real coordinates for this point are lat= 47.28755 lon= 11.63122
Closest ind: 810552
 Given lat: 47.305 vs found lat: 47.307
 Given lon: 11.622 vs found lon: 11.620
the grid index for Kolsass is 810552
the lowest level has a height of half level of 561.0
the lowest level has a height of full level of 571.0
the model topography is 561.0
the coordinates for this ind are lat= 47.30734 lon= 11.620166
the real coordinates for this point are lat= 47.305 lon= 11.622
Closest ind: 810583
 Given lat: 47.316 vs found lat: 47.316
 Given lon: 11.616 vs found lon: 11.617
the grid index for Eggen is 810583
the lowest level has a height of half level of 750.0
the 

In [5]:
###############################################################################################
## LOOP SETUP 2##
# time, loc, var
# loop 1 over time
for counter,pdate in enumerate(plotdates):
    ## READ IN ##
    lt = get_lt(pdate,simdate) # get leadtime
    filename = lfff_name(lt) # get filename
    nc_file = filepath+filename
    # If necessary, add the corresponding grid file:
    grid_file = '/store/s83/swester/teamx/grid_R19B08_size_cosmo1/d01_DOM01.nc'

    if iconvis.check_grid_information(nc_file):
        # print('             The grid information is available')
        data = psy.open_dataset(nc_file)
    else:
        print('             The grid information is not available')
        data = iconvis.combine_grid_information(nc_file,grid_file)


    print('calculating htd and ts '+pdate.strftime('%Y/%m/%d at %H:%M'))
    for loc in loc_list:
        print('     '+loc.name)
        # select variables from data dataset
        for pvar in pvars_list: # 3D vars for lidar
            # print(pvar.name)
            lidar_data[loc.name][pvar.name][counter] = data[pvar.name][0,:,loc.iconID].data.squeeze()[::-1] # reverse the variable arrays to have L 81 data at bottom
        for spvar in spvars_list: # 2D vars for lidar
            # print(pvar.name)
            ts_data[loc.name][spvar.name][counter] = data[spvar.name].data.squeeze()[loc.iconID]

calculating htd and ts 2019/09/12 at 12:00
     Hochhaueser
     Kolsass
     Eggen
     Weerberg
     Terfens
     Arbeser
     Radiosounding Munich
     Station & Radiosounding Innsbruck Airport
calculating htd and ts 2019/09/12 at 12:30
     Hochhaueser
     Kolsass
     Eggen
     Weerberg
     Terfens
     Arbeser
     Radiosounding Munich
     Station & Radiosounding Innsbruck Airport
calculating htd and ts 2019/09/12 at 13:00
     Hochhaueser
     Kolsass
     Eggen
     Weerberg
     Terfens
     Arbeser
     Radiosounding Munich
     Station & Radiosounding Innsbruck Airport
calculating htd and ts 2019/09/12 at 13:30
     Hochhaueser
     Kolsass
     Eggen
     Weerberg
     Terfens
     Arbeser
     Radiosounding Munich
     Station & Radiosounding Innsbruck Airport
calculating htd and ts 2019/09/12 at 14:00
     Hochhaueser
     Kolsass
     Eggen
     Weerberg
     Terfens
     Arbeser
     Radiosounding Munich
     Station & Radiosounding Innsbruck Airport
calculating htd

In [6]:
## CALCULATE NEW VARS AND SAVE ##
for loc in loc_list: # for each location
    # 3D cvars
    lidar_data[loc.name]['VEL'] = vf.calculate_wind_vel_from_uv(lidar_data[loc.name]['U'],lidar_data[loc.name]['V'])
    lidar_data[loc.name]['DIR'] = vf.calculate_wind_dir_from_uv(lidar_data[loc.name]['U'],lidar_data[loc.name]['V'],modulo_180=False)
    lidar_data[loc.name]['TH'] = calc.potential_temperature(lidar_data[loc.name]['P']* units.Pa,lidar_data[loc.name]['T']* units.kelvin)
    lidar_data[loc.name]['TD'] = calc.dewpoint_from_specific_humidity(lidar_data[loc.name]['P']* units.Pa, lidar_data[loc.name]['T']* units.kelvin, lidar_data[loc.name]['QV']* units('g/kg'))
    lidar_data[loc.name]['RH'] = calc.relative_humidity_from_specific_humidity(lidar_data[loc.name]['P']* units.Pa, lidar_data[loc.name]['T']* units.kelvin, lidar_data[loc.name]['QV']* units('g/kg'))
    lidar_data[loc.name]['P'] = lidar_data[loc.name]['P']/100
    
    # 2D scvars
    ts_data[loc.name]['VEL_10M'] = vf.calculate_wind_vel_from_uv(ts_data[loc.name]['u_10m'],ts_data[loc.name]['v_10m'])
    ts_data[loc.name]['DIR_10M'] = vf.calculate_wind_dir_from_uv(ts_data[loc.name]['u_10m'],ts_data[loc.name]['v_10m'],modulo_180=False)
    ts_data[loc.name]['T_2M'] = ts_data[loc.name]['T_2M']-273.15
    ts_data[loc.name]['TKE81'] = lidar_data[loc.name]['TKE'][:,80] # level 81
    ts_data[loc.name]['TKE80'] = lidar_data[loc.name]['TKE'][:,79] # level 80
    ts_data[loc.name]['TKE79'] = lidar_data[loc.name]['TKE'][:,78] # level 79
    ts_data[loc.name]['TKE78'] = lidar_data[loc.name]['TKE'][:,77] # level 78

    ## SAVE lidar/htd data ##
    np.save(savepath+'htd_ICON_'+loc.short+'.npy', lidar_data[loc.name]) 
    ## SAVE surface timeseries data ##
    np.save(savepath+'ts_ICON_'+loc.short+'.npy', ts_data[loc.name]) 

In [7]:
lidar_data[loc.name]['P'][0,:] 

array([961.86383207, 959.00973407, 955.1042343 , 950.45444759,
       945.22859894, 939.50815674, 933.34861531, 926.79108855,
       919.86837293, 912.60820412, 905.0341281 , 897.16514999,
       889.02006987, 880.61440903, 871.95652294, 863.05830347,
       853.93536038, 844.59787447, 835.0508952 , 825.29884684,
       815.34589407, 805.19580359, 794.85354681, 784.32404833,
       773.61086087, 762.71776422, 751.64652769, 740.40196759,
       728.98838241, 717.40587785, 705.6628771 , 693.77098607,
       681.74127092, 669.5916247 , 657.34339938, 645.00877471,
       632.58756404, 620.07517209, 607.46526507, 594.74715391,
       581.90998373, 568.94906997, 555.86632044, 542.66896455,
       529.36532891, 515.95809248, 502.44625772, 488.83719413,
       475.14001748, 461.36051872, 447.50630701, 433.58156539,
       419.58707934, 405.52120477, 391.38390357, 377.18424762,
       362.93720691, 348.65776259, 334.36732754, 320.08684344,
       305.81793345, 291.56023197, 277.31563351, 263.09